In [ ]:
import sys
import re
import os
import csv
import glob

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
def parse_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Extracting general details
    general_details = {}
    for i, line in enumerate(lines):
        if i < 9:
            if line.strip():
                key, value = line.split(maxsplit=1)
                general_details[key.strip()] = value.strip()
        else:
            break

    # Extracting hits table
    hits = []
    for line in lines[9:]:
        if not line.startswith("No"):
            # hit_info = line[:30].strip() + " " + line[30:].strip()
            if line.strip():
                hits.append(line)
        elif line.startswith("No"):
            break

    # Extracting full hit names
    full_hits = {}
    hit_details = ""
    hit_id = None
    for line in lines[9 + len(hits) + 1:]:
        if line.startswith("No"):
            hit_id = int(line.split()[1])
        elif line.startswith(">"):
            full_hits[hit_id] = line.strip()[1:]
    return general_details, hits, full_hits

def extract_non_overlapping_hits(hits, full_hits):
    def parse_coords(coords):
        start, end = coords.split('-')
        return int(start), int(end)

    non_overlapping_hits = []
    occupied_regions = []

    for hit in hits:
        # Assuming $line is the input string
        hit_parts = [hit[:3], hit[3:34], hit[34:40], hit[40:48], hit[48:56], hit[56:63], hit[63:69], hit[69:74], hit[74:84], hit[84:93], hit[93:]]
        hit_parts = [s.strip() for s in hit_parts]
        # print (hit_parts)

        # hit_parts = hit.split(" ",1)
        hit_id = int(hit_parts[0])
        query_coords = hit_parts[8]
        query_start, query_end = parse_coords(query_coords)

        overlap = False
        for start, end in occupied_regions:
            if not (query_end < start + 5 or query_start > end - 5):
                overlap = True
                break

        if not overlap:
            non_overlapping_hits.append({
                'No': hit_id,
                'Hit': hit_parts[1],
                'Prob': hit_parts[2],
                'Eval': hit_parts[3],
                'Cols': hit_parts[7],
                'Qf': query_start,
                'Qt': query_end,
                'Tf': hit_parts[9].split("-")[0],
                'Tt': hit_parts[9].split("-")[1],
                'FullName': full_hits[hit_id]
                
            })
            occupied_regions.append((query_start, query_end))

    return non_overlapping_hits

In [ ]:
# run on all proteins in the folder, create one table

# List to accumulate all non-overlapping hits for all proteins
all_hits = []

# Iterate over all files
for file_path in glob.iglob(f'/home/gridsan/lbrenes/hhpred/neighbors/mEp506_HicABC_system/hhsrch/*'): 
    base_name = os.path.basename(file_path)
    protein_name = os.path.splitext(base_name)[0]
    protein_name = os.path.splitext(protein_name)[0] # needed a second to remove additional suffix

    # Parse the file and extract hits
    general_details, hits, full_hits = parse_file(file_path)
    non_overlapping_hits = extract_non_overlapping_hits(hits, full_hits)

    # Plot system for visualization (if needed)
#    plot_system(non_overlapping_hits, int(general_details['Match_columns']))

    # Add protein name to each hit and accumulate all non-overlapping hits
    for hit in non_overlapping_hits:
        hit['Protein'] = protein_name  # Add protein name to the hit data
        all_hits.append(hit)

    # Save plot to seperate files
#    plt.savefig(f'../hhpred/IS_W0001_phage_diverse/parsed/figs/{protein_name}.svg', dpi=300, bbox_inches='tight')
#    plt.close()


# Convert the list of all hits to a DataFrame
keys_order = ['Protein', 'No', 'Hit', 'Prob', 'Eval', 'Cols', 'Qf', 'Qt', 'Tf', 'Tt', 'FullName']
df = pd.DataFrame(all_hits, columns=keys_order)

# Write the DataFrame to a single tab-delimited file
output_file_path = '~/hhpred/neighbors/mEp506_HicABC_system/parsed/mEp506_HicABC_system_neighbor_hits_summary.tsv' 
df.to_csv(output_file_path, sep='\t', index=False)


In [ ]:
HicABC_phrogs= pd.read_csv('/home/gridsan/lbrenes/hhpred/neighbors/mEp506_HicABC_system/parsed/mEp506_HicABC_system_neighbor_hits_summary.tsv',usecols=[0,1,2,3,4,5,6,7,8,9,10],names=['Protein', 'No', 'Hit', 'Prob', 'Eval', 'Cols', 'Qf', 'Qt', 'Tf', 'Tt', 'FullName'],sep='\t',skiprows=1)

In [ ]:
HicABC_phrogs_sorted=HicABC_phrogs.sort_values(by='Prob', ascending=False,ignore_index=False)

In [ ]:
HicABC_phrogs_sorted_nodups=HicABC_phrogs_sorted.drop_duplicates(subset="Protein",keep="first")

In [ ]:
HicABC_phrogs_sorted_nodups.to_csv('/home/gridsan/lbrenes/hhpred/neighbors/mEp506_HicABC_system/parsed/mEp506_HicABC_phrogs_cleaned.csv',index=False)

In [ ]:
HicABC_phrogs_NB= pd.read_csv('/home/gridsan/lbrenes/hhpred/neighbors/mEp506_HicABC_system/parsed/mEp506_HicABC_phrogs_cleaned.csv')

In [ ]:
HicABC_neighbors_FTs= pd.read_csv('/home/gridsan/lbrenes/hhpred/neighbors/mEp506_HicABC_system/mEp506_HicABC_neighbors_detailed_summary.csv')

In [ ]:
# Read the CSV files
HicABC_phrogs_NB = pd.read_csv('/home/gridsan/lbrenes/hhpred/neighbors/mEp506_HicABC_system/parsed/mEp506_HicABC_phrogs_cleaned.csv')
HicABC_neighbors_FTs = pd.read_csv('/home/gridsan/lbrenes/hhpred/neighbors/mEp506_HicABC_system/mEp506_HicABC_neighbors_detailed_summary.csv')

# Merge the DataFrames on the matching columns
merged_df = pd.merge(
    HicABC_neighbors_FTs,
    HicABC_phrogs_NB[['Protein','Prob', 'Eval', 'Cols','FullName']],  # Select only the necessary columns
    left_on='non-redundant_refseq',  # Column name in DUF_phrogs_NB
    right_on='Protein',  # Column name in PhiR5_1_neighbors_FTs
    how='left'  # Use 'left' join to keep all rows from DUF_phrogs_NB
)

# Drop the 'non-redundant refseq' column if not needed in the final DataFrame
merged_df.drop(columns=['Protein'], inplace=True)

merged_df.to_csv('/home/gridsan/lbrenes/hhpred/neighbors/mEp506_HicABC_system/parsed/mEp506_HicABC_phrogs_NB.csv', index=False)



### slight difference in hit #s are because non-redundant_refseq WPs that have been "suppressed" were not gathered by batch entrez for fasta info.
# it looks to be rather minor of a percentage tho

In [ ]:
# Load the data
mEp506_HicABC_phrogs_NB = pd.read_csv('/home/gridsan/lbrenes/hhpred/neighbors/mEp506_HicABC_system/parsed/mEp506_HicABC_phrogs_NB.csv')

def check_prophage(prob_series):
    total_count = len(prob_series)
    high_prob_count = (prob_series <= 0.0000000001).sum()
    if high_prob_count / total_count >= 0.15:
        return 1
    else:
        return 0

# Calculate the count of entries for each assembly
assembly_counts = mEp506_HicABC_phrogs_NB['assembly'].value_counts()

# Filter assemblies with at least 10 entries
valid_assemblies = assembly_counts[assembly_counts >= 10].index

# Filter the original DataFrame to include only valid assemblies
filtered_df = mEp506_HicABC_phrogs_NB[mEp506_HicABC_phrogs_NB['assembly'].isin(valid_assemblies)]

# Group by 'assembly' and apply the function to the 'Prob' column
result_df = filtered_df.groupby('assembly')['Eval'].apply(check_prophage).reset_index()

# Rename the column for clarity
result_df.rename(columns={'Prob': 'prophage'}, inplace=True)

# Save the resulting DataFrame

result_df.to_csv('/home/gridsan/lbrenes/hhpred/neighbors/mEp506_HicABC_system/parsed/mEp506_HicABC_prophage_hits.csv', index=False)
